In [2]:
import warnings
import sys
warnings.simplefilter('ignore')
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
from tqdm import tqdm 

In [3]:
import pandas as pd
bad = pd.read_csv('../data/clean/cleaned_bad.csv')
normal=pd.read_csv('../data/clean/cleaned_normal.csv')
# hiniku_=pd.read_csv('../data/hiniku/hiniku_with_word.csv')
hiniku = pd.read_csv("../data/clean/cleaned_hiniku.csv")
df = pd.read_csv('../data/clean/mount_all_data.csv')
# df.drop('comment_id', axis=1, inplace=True)

In [4]:
badwordlist = pd.read_csv('../data/clean/mount_bad_list.csv')
from janome.tokenizer import Tokenizer

tokenizer=Tokenizer()
badwordlist_reading = []
for word  in badwordlist.bad.values:
    text=""
    for t in tokenizer.tokenize(word):
        if t.reading=="*":
            pass
        else:
            text+=t.reading
    if len(text)>=1:
        badwordlist_reading.append(text)
# pd.DataFrame(badwordlist_reading).to_csv('../data/clean/badwordlist_reading.csv', index=False)

In [259]:
def ensemble(res1, res2, res3,threshold=0.5,display=True):
    train_ensemble = pd.DataFrame(np.array([res1["pred_train_un"],res2["pred_train_un"],res2["pred_train_un"]]).sum(axis=0)/3,columns=["pred"])
    test_ensemble = pd.DataFrame(np.array([res1["pred_test_un"],res2["pred_test_un"],res2["pred_test_un"]]).sum(axis=0)/3,columns=["pred"])
    train_ensemble = train_ensemble.pred.apply(lambda x: 1 if x>threshold else 0)
    test_ensemble = test_ensemble.pred.apply(lambda x: 1 if x>threshold else 0)
    train_score = f1_score(res1['y_train'], train_ensemble)
    test_score = f1_score(res1['y_test'], test_ensemble)
    
    miss_pred_tr = print_2x2_miss_pred(true_y=res1["y_train_untreat"], pred_y=train_ensemble, df=res1["x_train"],label="train")
    miss_pred_te = print_2x2_miss_pred(true_y=res1["y_test_untreat"], pred_y=test_ensemble, df=res1["x_test"],label="test")
#     coll_pred_tr = print_2x2_collect_pred(true_y=res1["y_train_untreat"], pred_y=train_ensemble, df=res1["x_train"])
#     coll_pred_te = print_2x2_collect_pred(true_y=res1["y_test_untreat"], pred_y=test_ensemble, df=res1["x_test"])
    coll_pred_tr = print_2x2_collect_pred(true_y=res1["y_train"], pred_y=train_ensemble, df=res1["x_train"],label="train")
    coll_pred_te = print_2x2_collect_pred(true_y=res1["y_test"], pred_y=test_ensemble, df=res1["x_test"],label="test")
    
    
    miss_pred_tr.pred_idx = miss_tr.pred_idx.apply(lambda x: int(x))
    miss_pred_te.pred_idx = miss_te.pred_idx.apply(lambda x: int(x))
    coll_pred_tr.pred_idx = coll_tr.pred_idx.apply(lambda x: int(x))
    coll_pred_te.pred_idx = coll_te.pred_idx.apply(lambda x: int(x))
    
    matrix_tr=np.array([coll_tr[coll_tr.pred_idx!=1].shape[0],
               miss_tr[miss_tr.pred_idx==1].shape[0],
               miss_tr[miss_tr.pred_idx!=1].shape[0], 
               coll_tr[coll_tr.pred_idx==1].shape[0]]).reshape(2,-1)
    
    matrix_te=np.array([coll_te[coll_te.pred_idx!=1].shape[0],
           miss_te[miss_te.pred_idx==1].shape[0],
           miss_te[miss_te.pred_idx!=1].shape[0], 
           coll_te[coll_te.pred_idx==1].shape[0]]).reshape(2,-1)
    if display:
        print("train:")
        print(train_score)
        print(matrix_tr)
        print('test:')
        print(test_score)
        print(matrix_te)
    
    return {"train_score":train_score,
            "test_score":test_score,
            "pred_train":train_ensemble,
            "pred_test":test_ensemble,
#             "miss_train_df":miss_pred_tr,
#             "miss_test_df":miss_pred_te,
#             "collect_train_df":coll_pred_tr,
#             "collect_test_df":coll_pred_te,
            "matrix_tr":matrix_tr,
            "matrix_te":matrix_te,
           }

In [27]:
from sklearn.svm import SVR
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


def train_and_pred(df, threshold=0.5, debug=False, random_state=42,display=True):
    df = df.dropna()
    ids = df.id.values
    X = df.drop(["target"],axis=1)#.drop('content', axis=1)
    y=df["target"]
    x_train, x_test, y_train_untreat, y_test_untreat = train_test_split(X,y, test_size=0.2, random_state=random_state)    
    
    y_train = y_train_untreat.apply(lambda x: 1 if x==1 else 0)#悪口
    y_test = y_test_untreat.apply(lambda x: 1 if x==1 else 0)

    tr_id = x_train.id.values
    x_train.drop('id', axis=1, inplace=True)
    te_id = x_test.id.values
    x_test.drop('id', axis=1, inplace=True)
    
    svm = SVR()
    svm.fit(x_train.drop(['content'], axis=1), y_train)
    pred_train_unadjust = svm.predict(x_train.drop(['content'], axis=1))
    pred_test_unadjust = svm.predict(x_test.drop(['content'], axis=1))
    pred_train = [1 if x>threshold else 0 for x in pred_train_unadjust]
    pred_test = [1 if x>threshold else 0 for x in pred_test_unadjust]
    
    if display:
        print("train")
        print("f1: " ,f1_score(y_train, pred_train))
        print(create_2x2_matrix(y_train, pred_train))
        # print_2x2_miss_pred(y_train, pred_train,x_train)
        print('---------------------------------')
        print('test')

        print("f1: ",f1_score(y_test, pred_test))
        print(create_2x2_matrix(y_test, pred_test))
        print('---------------------------------')
        print('---------------------------------')
    train_miss = print_2x2_miss_pred(true_y=y_train_untreat,pred_y=pred_train, label="train",df=x_train)
    test_miss =  print_2x2_miss_pred(true_y=y_test_untreat, pred_y=pred_test, label="test",df=x_test)
    train_coll = print_2x2_collect_pred(true_y=y_train_untreat,pred_y=pred_train, label="train",df=x_train)
    test_coll = print_2x2_collect_pred(true_y=y_test_untreat, pred_y=pred_test, label="test",df=x_test)
    df["id"]= ids
    return {
        "miss_train_df":train_miss, 
        "miss_test_df":test_miss,
        "collect_train_df":train_coll,
        "collect_test_df":test_coll,
        "x_train":x_train,
        "y_train":y_train,
        "x_test":x_test,
        "y_test":y_test,
        "pred_train":pred_train,
        "pred_test":pred_test,
        "model":svm,
        "pred_train_un":pred_train_unadjust,
        "pred_test_un":pred_test_unadjust,
        "df":df,
#         "train_id":tr_id,
#         "test_id":te_id,
        "y_train_untreat":y_train_untreat,
        "y_test_untreat":y_test_untreat
    }

In [7]:
def create_2x2_matrix(true_y, pred_y):
    matrix = np.zeros(4)

    for t,p in zip(true_y, pred_y):
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
        elif t==1:
            if p==0:matrix[2]+=1
            if p==1:matrix[3]+=1
    return matrix.reshape(2,2)


In [29]:
def print_2x2_miss_pred(true_y, pred_y,df, label,test=True):
    def print_str_label(num, phase="predict"):
        if phase=="predict":
            if num==0:return "その他"
            if num==1:return "悪口"
            else:
                return np.nan
            
        else:   
            if num==0:return "その他[普通]"
            if num==1:return "悪口"
            if num==2:return "その他[皮肉]"
            return np.nan

    idx_arr = []
    pred_judge=[]
    true_judge=[]
    col=["label","content","judge", "pred_idx", "pred_label","true_idx", "true_label"]
    returndf = pd.DataFrame(columns=col)
    for i in range(len(df)):
        if pred_y[i]!=true_y.values[i] and abs(pred_y[i]-true_y.values[i])!=2:
            tmpdf = pd.DataFrame(np.array([label,df.iloc[i].content,"miss", pred_y[i], print_str_label(pred_y[i],phase="predict"),\
                                            true_y.values[i], print_str_label(true_y.values[i], phase="")]).reshape(1,-1), \
                                              columns=col)
            returndf = pd.concat([returndf, tmpdf])
    return returndf


def print_2x2_collect_pred(true_y, pred_y,df,label, test=True):
    def print_str_label(num, phase):
        
        if phase=="predict":
            if num==0:return "その他"
            if num==1:return "悪口"
            else:
                return np.nan
            
        else:   
            if num==0:return "その他[普通]"
            if num==1:return "悪口"
            if num==2:return "その他[皮肉]"
            return np.nan
        
    idx_arr = []
    pred_judge=[]
    true_judge=[]
    col=["label","content", "judge","pred_idx", "pred_label","true_idx", "true_label"]
    returndf = pd.DataFrame(columns=col)
    for i in range(len(df)):
        if pred_y[i]==true_y.values[i]:
            tmpdf = pd.DataFrame(np.array([label,df.iloc[i].content,"collect", pred_y[i], print_str_label(pred_y[i],phase="predict"),\
                                            true_y.values[i], print_str_label(true_y.values[i], phase="")]).reshape(1,-1), \
                                              columns=col)
            returndf = pd.concat([returndf, tmpdf])
    return returndf

In [9]:
def create_wakati_content(text):
    wakati = []
    for t in tokenizer.tokenize(text):
        wakati.append(t.surface)
    return wakati

In [10]:
from gensim.models import Word2Vec
from tqdm import tqdm
# # data = pd.read_csv('../data/clean/mount_bad_list.csv')
# data = pd.read_csv("../data/clean/mount_all_data.csv")
# data = data[data.type==1]
wakati_arr = []
# for r in tqdm(data.iterrows()):
#     wakati = create_wakati_content(r[1].content)
#     wakati_arr.append(wakati)
# model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)


In [11]:
# data = pd.read_csv('../data/some_comment_data.csv')

# for r in tqdm(data.iterrows()):
#     wakati = create_wakati_content(r[1].content)
#     wakati_arr.append(wakati)
# model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)
# model.save("./word2vecModel/200000.model")


In [12]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load("./word2vecModel/100000.model")

In [13]:
'''
悪口ワードの含有
皮肉ワードの含有
ポジティブワードの含有

[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [形態素ごとに現れる悪口数]
[皮肉単語が含まれているかのフラグ],[形態素ごとに現れる皮肉数]
[ポジティブな単語が含まれているかのフラグ], [形態素ごとに現れるポジティブ単語数]

'''
import pandas as pd
# df = bad.copy()
# df = pd.concat([hiniku_, normal])
# df = pd.concat([bad, df])
import sys
import numpy as np
from janome.tokenizer import Tokenizer
from tqdm import tqdm
import gensim

df = pd.read_csv('../data/clean/mount_all_data.csv')
df.drop('comment_id', axis=1, inplace=True)
tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_word_count(text, wordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in wordlist:
            count+=1
    for w in wordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


def get_order_label(text):
    for t in tokenizer.tokenize(text):
        if t.infl_form[:2]=="命令":
            return 1
    return 0



def cos_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


'''
###ここを実行することでword2vecのモデル作成
from gensim.models import Word2Vec
data = pd.read_csv('../data/some_comment_data.csv')
newdata=pd.concat([pd.DataFrame(data.content), pd.DataFrame(df.content)])
wakati_arr = []
for r in tqdm(newdata.iterrows()):
    wakati = create_wakati_content(r[1].content)
    wakati_arr.append(wakadti)
model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)
'''

def calc_cosine_similary(text,model,debug=False):
    text_list = []
    scores = {}

    for t in tokenizer.tokenize(text):
        text_list.append(t.surface)
    for word in text_list:
        try:
            vector1 = model.wv[word]
            scores[word] = []
            for bad in bad_list:
                try:
                    vector2 = model.wv[bad]
                    score = cos_similarity(vector1,vector2)
                    scores[word].append(score)
                except :
                    pass
        except:
            pass
    all_scores = []
    for scores in scores.values():
        all_scores +=scores
    all_scores= np.array(all_scores)
    typicalV = np.median(all_scores, axis=0)

    if debug:
        return scores
    return typicalV


def mophologicize(text):
    words = []
    for t in tokenizer.tokenize(text):
        words.append(t.surface)    
    return words


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_features(df, badwordlist, positivelist=np.nan, hinikulist=np.nan):
#     col=["content","mophologics_num", "bad_per_mophologic", "simple_bad_flg","target", "hiniku_per_mophologic",\
#          "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
#     import gensim
#     PATH="./gensimmodel/word2vec.gensim.model"
#     model = gensim.models.Word2Vec.load(PATH)
    col=["content","mophologics_num", "bad_count_in_mophologic", "simple_bad_flg","target","order","mophologic_content"]

    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in tqdm(df.iterrows()):
        text = str(r[1].content)
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        badcount,badflg = get_word_count(text, badwordlist)
        orderlabel = get_order_label(text)
        mophologic = mophologicize(text)
#         hinikucount, hinikuflg = get_word_count(text, hinikulist)
#         positivecount, positiveflg = get_word_count(text, positivelist)
#         newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target,\
#                                        hinikucount, hinikuflg, positivecount,\
#                                        positiveflg]).reshape(1,-1),columns=col)
        newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target, orderlabel,mophologic]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
#     basedf["comment_id"] = basedf.index
    basedf["bad_count_in_mophologic"] = basedf.apply(lambda x: float(x["bad_count_in_mophologic"]), axis=1)
    basedf["mophologics_num"] = basedf.apply(lambda x: float(x["mophologics_num"]), axis=1)
    basedf["order"] = basedf.order.apply(lambda x: int(x))
#     basedf["cosine_similar"] = basedf.cosine_similar.apply(lambda x: float(x))
    
    return basedf
df =make_features(df, list(badwordlist.bad.values))
df = df.fillna(0.0)
# df = df[~df.duplicated(subset="content")]
df = df.reset_index().drop('index', axis=1)
df["id"] = df.index
display(df.head(2))
print(df.shape)


1929it [00:15, 122.84it/s]


,content,mophologics_num,bad_count_in_mophologic,simple_bad_flg,target,order,mophologic_content,id
0,なにこの糞ゲー,4.0,1.0,1,1,0,"[なに, この, 糞, ゲー]",0
1,ちょっと蛇足気味だな・・・,8.0,1.0,1,1,0,"[ちょっと, 蛇足, 気味, だ, な, ・, ・, ・]",1


(1929, 8)


In [14]:
def create_bad_word_list():
    b_l=[]
    #wor2vecの辞書にない単語の除外
    for word in badwordlist.bad.values:
        try:
            a = model.wv[word]
            b_l.append(word)
        except:
            pass
    return b_l

In [15]:

col=[]
for i in range(100):
    col.append("f_"+str(i))
def create_mean_vector_data(df):

    sandbox=pd.DataFrame(columns=col)
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        vec_list=[]
        for t in text:
                try:
                    vec = model.wv[t]
                    vec_list.append(vec)
                except:
                    vec_list.append(np.zeros(100))
        vectors = pd.DataFrame(np.array(vec_list), columns=col)
        tmp = pd.DataFrame(pd.DataFrame(vectors, columns=col).mean(axis=0)).T
        sandbox = pd.concat([sandbox, tmp])
        

    sandbox.insert(0, "target",df.target.values)
    sandbox.insert(1,"content",df.content.values)
    sandbox["id"]= df.id.values
    return sandbox

In [16]:
def create_mean_cosine_sim_data(df):
    b_l=create_bad_word_list()
    base = pd.DataFrame(columns=np.array(b_l))
    
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        ave_list=[]

        for word in b_l:
            try:
                v1 = model.wv[word]
                cos_list = []
                for t in text:
                        v2=model.wv[t]
                        cos_list.append(cos_similarity(v1,v2))
                        if word not in b_l:
                            b_l.append(word)
            except:
                pass
            ave_list.append(np.array(cos_list).mean())
        tmp=pd.DataFrame(np.array(ave_list).reshape(1,-1), columns=b_l)
        base=pd.concat([base, tmp])
        
    base["target"] =df.target.values
    base["content"] = df.content.values
    base = base.loc[:,base.columns[::-1]]
    base["id"] = df.id.values
    return base

In [17]:
def create_the_highest_cosine_sim_data(df):
    b_l = create_bad_word_list()
    base = pd.DataFrame(columns=np.array(b_l))
    
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        highest_list=[]

        for word in b_l:
            try:
                v1 = model.wv[word]
                cos_list = []
                for t in text:
                        v2=model.wv[t]
                        cos_list.append(cos_similarity(v1,v2))
                        if word not in b_l:
                            b_l.append(word)
            except:
                pass
            try:
                highest_list.append(np.array(cos_list).max())
            except:
                highest_list.append(np.nan)
        tmp=pd.DataFrame(np.array(highest_list).reshape(1,-1), columns=b_l)
        base=pd.concat([base, tmp])
        
    base["target"] =df.target.values
    base["content"] = df.content.values
    base = base.loc[:,base.columns[::-1]]
    base["id"] = df.id.values
    return base


# high_sim_df.head(2)

In [30]:
df1 = create_mean_vector_data(df)

1929it [00:06, 303.51it/s]


In [31]:
res1 = train_and_pred(df1)

train
f1:  0.7585743106926699
[[620. 175.]
 [184. 564.]]
---------------------------------
test
f1:  0.6704225352112677
[[150.  52.]
 [ 65. 119.]]
---------------------------------
---------------------------------


In [32]:
df2 = create_mean_cosine_sim_data(df)
res2 = train_and_pred(df2,threshold=0.5)

1929it [00:50, 37.91it/s]


train
f1:  0.636429085673146
[[545. 209.]
 [296. 442.]]
---------------------------------
test
f1:  0.6382978723404256
[[118.  62.]
 [ 74. 120.]]
---------------------------------
---------------------------------


In [35]:
df3 = create_the_highest_cosine_sim_data(df)
res3 = train_and_pred(df3,threshold=0.5)

1929it [00:54, 35.09it/s]


train
f1:  0.6666666666666666
[[566. 188.]
 [275. 463.]]
---------------------------------
test
f1:  0.6892655367231638
[[142.  38.]
 [ 72. 122.]]
---------------------------------
---------------------------------


In [36]:
df1.shape, df2.shape, df3.shape

((1929, 103), (1929, 155), (1929, 155))

In [37]:
res1["df"].shape,res2["df"].shape,res3["df"].shape

((1929, 103), (1866, 155), (1866, 155))

In [38]:
valid_id = list(res2["df"].id.values)

In [77]:
newdf = pd.DataFrame(columns=df.columns)
for i in tqdm(range(len(df))):
    if i in valid_id:
        tmp = pd.DataFrame(df.iloc[i]).T
        newdf = pd.concat([newdf, tmp])

100%|██████████| 1929/1929 [00:05<00:00, 370.70it/s]


In [84]:
# for r in newdf.iterrows():
#     if "w" in r[1].content:
#         r[1].content = r[1].content.replace("w", "")+"w"
#     if " " in r[1].content:
#         r[1].content = r[1].content.replace(' ', "")
#     if "　" in r[1].content:
#         r[1].content = r[1].content.replace('　', "")

In [42]:
df1=create_mean_vector_data(newdf)
df2=create_the_highest_cosine_sim_data(newdf)
df3=create_mean_cosine_sim_data(newdf)

1866it [00:07, 242.30it/s]
1866it [00:43, 43.11it/s]
1866it [00:54, 34.11it/s]


In [253]:
res1 = train_and_pred(df1,)
res2 = train_and_pred(df2,)
res3 = train_and_pred(df3,)

In [260]:
res=ensemble(res1,res2,res3)

train:
0.7402689313517339
[[602 152]
 [215 523]]
test:
0.7645429362880886
[[151  29]
 [ 56 138]]


In [236]:
# df1=create_mean_vector_data(newdf)
# df2=create_the_highest_cosine_sim_data(newdf)
# df3=create_mean_cosine_sim_data(newdf)
# res1 = train_and_pred(df1, display=False)
# res2 = train_and_pred(df2, display=False)
# res3 = train_and_pred(df3, display=False)

In [249]:

def change_seed(df1,df2,df3,show=False, iterations=5):
    trains=[]
    tests =[]

    for i in tqdm(range(iterations)):
        res1 = train_and_pred(df1,display=show)
        res2 = train_and_pred(df2,display=show)
        res3 = train_and_pred(df3,display=show)
        res=ensemble(res1,res2,res3,display=show)
        trains.append(res["train_score"])
        tests.append(res["test_score"])

    trains = np.array(trains)
    tests = np.array(tests)
    if show:
        print(trains.min(), trains.max(), trains.mean(),trains.var(ddof=-1))
        print(tests.min(), tests.max(), tests.mean(),tests.var(ddof=-1))
    return trains, tests

In [250]:
tr, te = change_seed(df1, df2, df3)

100%|██████████| 5/5 [00:59<00:00, 11.88s/it]


In [252]:
print(tr.min(), tr.max(), tr.mean(),tr.var(ddof=-1))
print(te.min(), te.max(), te.mean(),te.var(ddof=-1))

0.7402689313517339 0.7402689313517339 0.7402689313517339 0.0
0.7645429362880886 0.7645429362880886 0.7645429362880886 0.0


In [205]:
miss_tr=res["miss_train_df"]
miss_te=res["miss_test_df"]
coll_tr=res["collect_train_df"]
coll_te=res["collect_test_df"]

In [254]:
miss_tr.to_csv('./predicted_data/miss_pred_train.csv', index=False)
miss_te.to_csv('./predicted_data/miss_pred_test.csv', index=False)
coll_tr.to_csv('./predicted_data/collect_pred_train.csv', index=False)
coll_te.to_csv("./predicted_data/collect_pred_test.csv", index=False)

In [262]:
print(res["matrix_tr"])
print(res["matrix_te"])

[[602 152]
 [215 523]]
[[151  29]
 [ 56 138]]


In [270]:
# #悪口と誤判定してしまった
# res["miss_train_df"].true_idx.value_counts()#普通 : 皮肉 = 83 ; 59
# tmp = res["miss_train_df"][res["miss_train_df"].pred_idx=="1"]#
# print(tmp[tmp.true_idx=="2"].shape)#悪口と判定された皮肉
# tmp[tmp.true_idx=="2"]

In [156]:
a = res["miss_train_df"]
b = res["miss_test_df"]
c = res["collect_train_df"]
d = res["collect_test_df"]

# train_df = pd.concat([a,c])
# test_df = pd.concat([b,d])

# predict_df = pd.concat([a,b,c,d])
# train_df.to_csv('./predicted_data/train.csv', index=False)
# test_df.to_csv('./predicted_data/test.csv',index=False)
# predict_df.to_csv("./predicted_data/merged_train_test.csv",index=False)

In [29]:
# testdf = base.copy()
# testdf = testdf.dropna()
# # testdf.target = testdf.target.apply(lambda x: 0 if x==0 else 1)#x==0(ニュートラル)の時0, その他は1
# testdf.target = testdf.target.apply(lambda x: 0 if x==1 else 1)#x==1(悪口)の時0, その他は1 [0:悪口, 1:その他]
# # testdf.target = testdf.target.apply(lambda x: 0 if x==2 else 1)#x==2(皮肉)の時0, その他は1
# testdf.target.unique()
# testdf.target.unique()
# X = testdf.drop("target",axis=1)#.drop('content', axis=1)
# y=testdf.target


In [164]:
res["matrix_tr"]

array([[602., 152.],
       [215., 523.]])

In [162]:
a.shape[0]+ b.shape[0]+ c.shape[0]+d.shape[0]

1866

In [32]:
"""
1対その他での学習
"""


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



def train_and_predict_one_vs_rest(df):
    
    X = df.drop(["target","id"],axis=1)
    y = df.target
    

    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
#     return x_train, y_train

    train_content = pd.DataFrame(x_train.content.values, columns=["content"])
    test_content  = pd.DataFrame(x_test.content.values, columns=["content"])
#     x_train.drop(,axis=1,inplace=True)
#     x_test.drop(["content"],axis=1,inplace=True)
    svm = SVC()
    ovr = OneVsRestClassifier(svm)
    ovr.fit(x_train.drop('content',axis=1), list(y_train.values))
    
    tr_pre = ovr.predict(x_train.drop('content',axis=1))
    te_pre = ovr.predict(x_test.drop('content',axis=1))
    
    
    

    return {
#         "pred_train_with_vec":  vec_model.predict(new_x_train),
#         "pred_test_with_vec": vec_model.predict(new_x_test),
        "pred_train":ovr.predict(x_train.drop('content',axis=1)),
        "pred_test":ovr.predict(x_test.drop('content',axis=1)),
        "x_train":x_train,
#         "new_x_train":new_x_train,
        "y_train":y_train,
#         "new_x_test":new_x_test,
        "x_test":x_test,
        "y_test":y_test,
        "train_content":train_content,
        "test_content":test_content,
    }

In [284]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import numpy as np
'''
mxm,ここでは3x3のマトリックスを作成する
そこからf1値を計算する（同ディレクトリの画像参照）
'''

def create_3x3_matrix(true_y, pred_y):
    matrix = np.zeros(9)

    for t,p in zip(true_y, pred_y):
        t = int(t)
        p = int(p)
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
            if p==2:matrix[2]+=1
        elif t==1:
            if p==0:matrix[3]+=1
            if p==1:matrix[4]+=1
            if p==2:matrix[5]+=1
        elif t==2:
            if p==0:matrix[6]+=1
            if p==1:matrix[7]+=1
            if p==2:matrix[8]+=1

    return matrix.reshape(3,3)
# matrix=create_3x3_matrix(y_train, pred_train)
# matrix

In [34]:

# matrix

'''
matrix = np.array([
    [240,20, 40],
    [5,180, 15],
    [25, 5, 70]
])
動作例
acc: 0.8000000000000002
recall 0.7756458897922313
f1: 0.7876347291657112
'''
def calc_3x3_matrix_f_score(matrix):

    accs = []
    recalls=[]
    tate=matrix.sum(axis=0)#たて
    yoko=matrix.sum(axis=1)#横
    for i in range(3):
        accs.append(matrix[i][i]/yoko[i])
        recalls.append(matrix[i][i]/tate[i])
    acc = np.mean(accs)
    recall = np.mean(recalls)

    f1 = 2*acc*recall/(acc+recall)
    return {
        "acc":acc,
        "recall":recall,
        "f1":f1,
    }

In [35]:
# df1[df1.target==0].shape[0], df1[df1.target==1].shape[0],df1[df1.target==2].shape[0]

In [36]:
'''
predのキー
dict_keys(['pred_train', 'pred_test', 'x_train', 'new_x_train', 'y_train',\
'new_x_test', 'x_test', 'y_test', 'train_content', 'test_content'])
'''

def print_3x3miss_pred(pred, test=True, true_y=np.nan, pred_y=np.nan,df=np.nan):
    if test:
        true_y = pred["y_test"]
        pred_y = pred["pred_test"]
        df     = pred["test_content"]
    else:
        true_y = pred["y_train"]
        pred_y = pred["pred_train"]
        df     = pred["train_content"]


    idx_arr = []
    pred_judge=[]
    true_judge=[]
    for i in range(len(pred_y)):
        if pred_y[i]!=true_y.values[i]:
#             print(pred[i], true_y.values[i])
            idx_arr.append(i)
            pred_judge.append(pred_y[i])
            true_judge.append(true_y.values[i])

    def print_str_label(num):
        if num==0:return "ニュートラル"
        if num==1:return "悪口"
        if num==2:return "皮肉"
        return np.nan
    
    col=["content","true_label_id","true_label","pred_label_id","pred_label"]
    miss_data = pd.DataFrame(columns=col)
    for i,idx in enumerate(idx_arr):
        tmpdf = pd.DataFrame(np.array([df.iloc[idx].content, true_judge[i],\
                                          print_str_label(true_judge[i]), pred_judge[i],\
                                        print_str_label(pred_judge[i])]).reshape(1,-1),columns=col)
        miss_data = pd.concat([miss_data, tmpdf])
        
#             display(df.iloc[idx].content)
#             print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
#             print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
#             print("------------------------------------------------------------------------------------")
        
    print(miss_data.shape)
    return miss_data

In [38]:
#df1 create_mean_vector_data
#df2 create_mean_cosine_sim_data
#df3 createcreate_the_highest_cosine_sim_data

In [39]:
# OneVsRest= train_and_predict_one_vs_rest(df1)
# matrix_tr=create_3x3_matrix(true_y=OneVsRest["y_train"], pred_y=OneVsRest["pred_train"])
# matrix_te=create_3x3_matrix(true_y=OneVsRest["y_test"], pred_y=OneVsRest["pred_test"])
# print("train:",calc_3x3_matrix_f_score(matrix_tr))
# print(matrix_tr)
# print("test:",calc_3x3_matrix_f_score(matrix_te))
# print(matrix_te)

train: {'acc': 0.4988334641742502, 'recall': 0.7800663055590945, 'f1': 0.6085280280702309}
[[309. 244.   0.]
 [ 79. 659.   0.]
 [ 46. 146.   9.]]
test: {'acc': 0.4705326460481099, 'recall': 0.5290123456790123, 'f1': 0.4980617798391572}
[[ 70.  69.   1.]
 [ 20. 172.   2.]
 [ 10.  29.   1.]]


In [40]:
# OneVsRest= train_and_predict_one_vs_rest(df2)
# matrix_tr=create_3x3_matrix(true_y=OneVsRest["y_train"], pred_y=OneVsRest["pred_train"])
# matrix_te=create_3x3_matrix(true_y=OneVsRest["y_test"], pred_y=OneVsRest["pred_test"])
# print("train:",calc_3x3_matrix_f_score(matrix_tr))
# print(matrix_tr)
# print("test:",calc_3x3_matrix_f_score(matrix_te))
# print(matrix_te)

train: {'acc': 0.44383922139402227, 'recall': nan, 'f1': nan}
[[271. 282.   0.]
 [117. 621.   0.]
 [ 61. 140.   0.]]
test: {'acc': 0.41605301914580267, 'recall': nan, 'f1': nan}
[[ 60.  80.   0.]
 [ 35. 159.   0.]
 [ 13.  27.   0.]]


In [41]:
# OneVsRest= train_and_predict_one_vs_rest(df3)
# matrix_tr=create_3x3_matrix(true_y=OneVsRest["y_train"], pred_y=OneVsRest["pred_train"])
# matrix_te=create_3x3_matrix(true_y=OneVsRest["y_test"], pred_y=OneVsRest["pred_test"])
# print("train:",calc_3x3_matrix_f_score(matrix_tr))
# print(matrix_tr)
# print("test:",calc_3x3_matrix_f_score(matrix_te))
# print(matrix_te)

train: {'acc': 0.4185213394062634, 'recall': 0.5800970795763215, 'f1': 0.48623779036100034}
[[214. 337.   2.]
 [108. 630.   0.]
 [ 50. 148.   3.]]
test: {'acc': 0.3930289641629848, 'recall': nan, 'f1': nan}
[[ 46.  94.   0.]
 [ 29. 165.   0.]
 [ 14.  26.   0.]]
